<center>
<img src="https://drive.google.com/uc?id=1f1gGVI-rxcHjA90WEGNvvtSXF1pAxQwg" alt="Fasilkom UI" width="300"/>

CSCE604137 • Penambangan Data

Semester Ganjil 2021/2022

Fakultas Ilmu Komputer, Universitas Indonesia

##**Tugas Kelompok 1**

###**Tenggat Waktu: 15 April 2022, 22.00 WIB**
</center>

####**Ketentuan:**

Anggota Kelompok:
- Amanda Carrisa Ashardian - 1906696969
- Muhamad Yoga Mahendra - 1806205256
- Nabila Fathia Zahra - 1706022741


In [1]:
# Import
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import plotly.express as px
import scipy.stats as scp
import seaborn as sn
import sklearn as sk
import statsmodels.api as sm
from scipy.linalg import eigh
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import *
from sklearn.cluster import AgglomerativeClustering, KMeans
from sklearn.metrics import pairwise, silhouette_score
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV, LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.metrics import roc_curve
from sklearn.metrics import confusion_matrix
!pip install PySastrawi

In [2]:
from sklearn.metrics import accuracy_score, f1_score


In [3]:
print("apalah")

apalah


In [4]:
# from google.colab import drive
# drive.mount('/content/drive')

In [5]:
# # Import dataset dengan index di kolom pertama
# cancer_reg = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/KASDD/TP1/cancer_reg.csv', index_col=0)
# startup_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/KASDD/TP1/startup_data.csv', index_col=0)

In [6]:
# Helper Function Definitions
def cek_null(df):
    col_na = df.isnull().sum().sort_values(ascending=False)
    percent = col_na / len(df)
    
    missing_data = pd.concat([col_na, percent], axis=1, keys=['Total', 'Percent'])
    print(missing_data[missing_data['Total'] > 0])
  
def count_outliers(df, col):
    outliers = 0
    q1 = df[col].quantile(.25)
    q3 = df[col].quantile(.75)
    iqr = q3 - q1
    w = 1.5 * iqr

    for i in df.index:
        row = df[col][i]
        if (row < (q1 - w) or row > (q3 + w)):
            outliers += 1
    return outliers

TO DO List:
- Preprosesing data (ubah jadi tabular/csv)
- feature exploration (jadiin bow/word embedding)
- Implement naive bayes/multilayer/smth
- cek hasil pake scoring macro micro F1 scoring, hamming loss, exact match ratio
- pake supervised learning 
- abis itu ensembled (N model digabung/agregat terus divote yang mana yg paling banyak)

Pembagian:

Amanda:
macro micro F1 scoring, hamming loss

Thia:
Preprosesing data,
feature exploration

Yoga:
exact match ratio,
bantu manda thia


In [7]:
# Converting csv to dataframe
# amanda
# contoh = pd.read_csv('/Data TK 1 Text Mining/clean/contoh_data.csv')
# human_ant = pd.read_csv('/Data TK 1 Text Mining/clean/human_annotated.csv')
# machine_ant = pd.read_csv('/Data TK 1 Text Mining/clean/machine_annotated.csv')
# testing = pd.read_csv('/Data TK 1 Text Mining/clean/testing_data.csv')
# unlabeled = pd.read_csv('/Data TK 1 Text Mining/clean/unlabeled_data.csv')

# amanda
contoh = pd.read_csv('contoh_data.csv')
human_ant = pd.read_csv('human_annotated.csv')
machine_ant = pd.read_csv('machine_annotated.csv')
testing = pd.read_csv('testing_data.csv')
unlabeled = pd.read_csv('unlabeled_data.csv')


# # yoga
# contoh = pd.read_csv('/Colab Notebooks/Data Mining/Data TK 1 Text Mining/clean/contoh_data.csv')
# human_ant = pd.read_csv('/Colab Notebooks/Data Mining/Data TK 1 Text Mining/clean/human_annotated.csv')
# machine_ant = pd.read_csv('/Colab Notebooks/Data Mining/Data TK 1 Text Mining/clean/machine_annotated.csv')
# testing = pd.read_csv('/Colab Notebooks/Data Mining/Data TK 1 Text Mining/clean/testing_data.csv')
# unlabeled = pd.read_csv('/Colab Notebooks/Data Mining/Data TK 1 Text Mining/clean/unlabeled_data.csv')

In [8]:
# Previews
print("Preview contoh_data")
contoh.head()

Preview contoh_data


,id,label_0,label_1,label_2,judul,isi
0,AD-4081,Umum,Penyakit Dalam,NaN,nyeri ulu hati,dok kemarin saya mkam makanan pedas nah sehari...
1,AD-4086,Kesehatan Anak,NaN,NaN,bayi susah makan ketika tumbuh gigi,malam dok . saya punya seorang bayi usia 7 bul...
2,AD-4095,Kesehatan Kulit dan Kelamin,Penyakit Dalam,Umum,kulit wajah berminyak dan sensitif,"hallo dok , saya baru berusia 17 tahun . saya ..."
3,AD-4373,Kesehatan Gizi,Penyakit Dalam,NaN,cara menambah berat badan dan obatnya,"siang dok , cara nambahin berat badan gimana c..."
4,AD-4386,Ortopedi (Tulang),NaN,NaN,cara menyembuhkan patah tulang di selakangan,"dok , kaki saya patah di selakangan akibat ber..."


In [9]:
print("Preview human_annotated_data")
human_ant.head()

Preview human_annotated_data


,id,label_0,label_1,label_2,judul,isi
0,DS-1,Umum,Kesehatan Kulit dan Kelamin,NaN,mengapa keringat badan sangat berlebihan ?,selamat malam dokter. ! saya fathurrosi umur 1...
1,DS-23,Umum,Kesehatan Jiwa,NaN,suka menghayal dan berhalusinasi,"selamat siang dokter , saya sering kali berkha..."
2,DS-87,Umum,Penyakit Dalam,Bedah,feses warna kuning di sertai darah apakah ada ...,"selamat sore dokter , feses saya warna kuning ..."
3,DS-176,Umum,Bedah,NaN,apakah setiap nyeri perut kanan bawah itu adal...,"selamat sore dokter , saya merasakan nyeri per..."
4,DS-277,Umum,Pulmonologi (Paru),NaN,10 kebiasaan buruk yang dapat merusak otak,"saya perokok berat dok , mengenal rokok sejak ..."


In [10]:
machine_ant

,id,label_0,label_1,label_2,Unnamed: 4,Unnamed: 5,label_3,label_4,label_5,label_6,label_7,label_8,judul,isi
0,AD-10,Kesehatan Kulit dan Kelamin,Umum,Urologi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,gatal-gatal dan bintik merah di kelamin,dok saya cowok ko alat kelamin saya gatal ya ....
1,AD-15,Bedah,Urologi,Penyakit Dalam,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sering buang air kecil,selamat siang dok . sudah beberapa minggu ini ...
2,AD-28,Jantung dan Pembuluh Darah,Penyakit Dalam,Umum,NaN,NaN,Bedah,NaN,NaN,NaN,NaN,NaN,penyebab nyeri dada,assalammu alaikum . salam sejahtera bagi kita ...
3,AD-33,Penyakit Dalam,Saraf,Kesehatan Anak,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,penanganan luka digigit anjing,"malam dok , tadi saya digigit anjing tetapi an..."
4,AD-36,Kesehatan Jiwa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,insomnia karena banyak pikiran,"malam dok , sebelum nya knlin dok nma saya riw..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11521,KD-43440,Kesehatan Mata,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mata berwarna kuning,"dok , mata saya bagian yang putihnya itu berwa..."
11522,KD-43442,Gigi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,behel pada orang dewasa,"selamat siang dok , mau tanya apakah masih bis..."
11523,KD-43451,Gigi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,gigi geraham bungsu tidak kunjung tumbuh sempurna,"mau tanya dok , kira-kira dibutuhkan berapa la..."
11524,KD-43460,Kesehatan Jiwa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,berjalan dalam tidur,dok saya ingin bertanya akhir-akhir ini saya m...


In [11]:
machine_ant.append(human_ant, ignore_index = True)

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


,Unnamed: 4,Unnamed: 5,id,isi,judul,label_0,label_1,label_2,label_3,label_4,label_5,label_6,label_7,label_8
0,NaN,NaN,AD-10,dok saya cowok ko alat kelamin saya gatal ya ....,gatal-gatal dan bintik merah di kelamin,Kesehatan Kulit dan Kelamin,Umum,Urologi,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,AD-15,selamat siang dok . sudah beberapa minggu ini ...,sering buang air kecil,Bedah,Urologi,Penyakit Dalam,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,AD-28,assalammu alaikum . salam sejahtera bagi kita ...,penyebab nyeri dada,Jantung dan Pembuluh Darah,Penyakit Dalam,Umum,Bedah,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,AD-33,"malam dok , tadi saya digigit anjing tetapi an...",penanganan luka digigit anjing,Penyakit Dalam,Saraf,Kesehatan Anak,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,AD-36,"malam dok , sebelum nya knlin dok nma saya riw...",insomnia karena banyak pikiran,Kesehatan Jiwa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13296,NaN,NaN,KD-43177,"malam dok , saya ingin tahu apakah penyakit us...",usus buntu,Umum,Bedah,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13297,NaN,NaN,KD-43189,selamat siang dokter saya mau konsultasi soal ...,telinga berdenging,Umum,"Kesehatan Telinga, Hidung dan Tenggorokan (THT)",NaN,NaN,NaN,NaN,NaN,NaN,NaN
13298,NaN,NaN,KD-43353,salam sehat dok untuk menjaga kesehatan selain...,aktivitas berat ?,Umum,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13299,NaN,NaN,KD-43435,"dok , saya mau tanya seputar penyakit liver.ka...",penyakit liver,Kesehatan Gizi,Penyakit Dalam,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
machine_ant["isi"] = machine_ant["judul"] + machine_ant["isi"]

In [13]:
machine_ant

,id,label_0,label_1,label_2,Unnamed: 4,Unnamed: 5,label_3,label_4,label_5,label_6,label_7,label_8,judul,isi
0,AD-10,Kesehatan Kulit dan Kelamin,Umum,Urologi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,gatal-gatal dan bintik merah di kelamin,gatal-gatal dan bintik merah di kelamindok say...
1,AD-15,Bedah,Urologi,Penyakit Dalam,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sering buang air kecil,sering buang air kecilselamat siang dok . suda...
2,AD-28,Jantung dan Pembuluh Darah,Penyakit Dalam,Umum,NaN,NaN,Bedah,NaN,NaN,NaN,NaN,NaN,penyebab nyeri dada,penyebab nyeri dadaassalammu alaikum . salam s...
3,AD-33,Penyakit Dalam,Saraf,Kesehatan Anak,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,penanganan luka digigit anjing,"penanganan luka digigit anjingmalam dok , tadi..."
4,AD-36,Kesehatan Jiwa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,insomnia karena banyak pikiran,"insomnia karena banyak pikiranmalam dok , sebe..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11521,KD-43440,Kesehatan Mata,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mata berwarna kuning,"mata berwarna kuningdok , mata saya bagian yan..."
11522,KD-43442,Gigi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,behel pada orang dewasa,"behel pada orang dewasaselamat siang dok , mau..."
11523,KD-43451,Gigi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,gigi geraham bungsu tidak kunjung tumbuh sempurna,gigi geraham bungsu tidak kunjung tumbuh sempu...
11524,KD-43460,Kesehatan Jiwa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,berjalan dalam tidur,berjalan dalam tidurdok saya ingin bertanya ak...


In [14]:
KebidananArray = []
for index, row in machine_ant.iterrows():
  if row['label_0'] == "Kebidanan dan Kandungan" or row['label_1'] == "Kebidanan dan Kandungan" or row['label_2'] == "Kebidanan dan Kandungan"or row['label_3'] == "Kebidanan dan Kandungan"or row['label_4'] == "Kebidanan dan Kandungan"or row['label_5'] == "Kebidanan dan Kandungan"or row['label_6'] == "Kebidanan dan Kandungan"or row['label_7'] == "Kebidanan dan Kandungan"or row['label_8'] == "Kebidanan dan Kandungan":
    KebidananArray.append("1")
  else:
    KebidananArray.append("0")
print(len(KebidananArray))



11526


In [15]:
DalamArray = []
for index, row in machine_ant.iterrows():
  if row['label_0'] == "Penyakit Dalam" or row['label_1'] == "Penyakit Dalam" or row['label_2'] == "Penyakit Dalam" or row['label_3'] == "Penyakit Dalam" or row['label_4'] == "Penyakit Dalam" or row['label_5'] == "Penyakit Dalam" or row['label_6'] == "Penyakit Dalam" or row['label_7'] == "Penyakit Dalam" or row['label_8'] == "Penyakit Dalam":
    DalamArray.append("1")
  else:
    DalamArray.append("0")
print(len(DalamArray))


11526


In [16]:
AnakArray = []
for index, row in machine_ant.iterrows():
  if row['label_0'] == "Kesehatan Anak" or row['label_1'] == "Kesehatan Anak" or row['label_2'] == "Kesehatan Anak" or row['label_3'] == "Kesehatan Anak" or row['label_4'] == "Kesehatan Anak" or row['label_5'] == "Kesehatan Anak" or row['label_6'] == "Kesehatan Anak" or row['label_7'] == "Kesehatan Anak" or row['label_8'] == "Kesehatan Anak":
    AnakArray.append("1")
  else:
    AnakArray.append("0")
print(len(AnakArray))
print(AnakArray[0])


11526
0


In [17]:
KulitArray = []
for index, row in machine_ant.iterrows():
  if row['label_0'] == "Kesehatan Kulit dan Kelamin" or row['label_1'] == "Kesehatan Kulit dan Kelamin" or row['label_2'] == "Kesehatan Kulit dan Kelamin" or row['label_3'] == "Kesehatan Kulit dan Kelamin" or row['label_4'] == "Kesehatan Kulit dan Kelamin" or row['label_5'] == "Kesehatan Kulit dan Kelamin"or row['label_6'] == "Kesehatan Kulit dan Kelamin" or row['label_7'] == "Kesehatan Kulit dan Kelamin" or row['label_8'] == "Kesehatan Kulit dan Kelamin":
    KulitArray.append("1")
  else:
    KulitArray.append("0")
print(len(KulitArray))
print(KulitArray[0])


11526
1


In [18]:
GiziArray = []
for index, row in machine_ant.iterrows():
  if row['label_0'] == "Kesehatan Gizi" or row['label_1'] == "Kesehatan Gizi" or row['label_2'] == "Kesehatan Gizi"or row['label_3'] == "Kesehatan Gizi" or row['label_4'] == "Kesehatan Gizi"or row['label_5'] == "Kesehatan Gizi" or row['label_6'] == "Kesehatan Gizi"or row['label_7'] == "Kesehatan Gizi" or row['label_8'] == "Kesehatan Gizi":
    GiziArray.append("1")
  else:
    GiziArray.append("0")
print(len(GiziArray)) 

11526


In [19]:
THTArray = []
for index, row in machine_ant.iterrows():
  if row['label_0'] == "Kesehatan Telinga, Hidung dan Tenggorokan (THT)" or row['label_1'] == "Kesehatan Telinga, Hidung dan Tenggorokan (THT)" or row['label_2'] == "Kesehatan Telinga, Hidung dan Tenggorokan (THT)"or row['label_3'] == "Kesehatan Telinga, Hidung dan Tenggorokan (THT)" or row['label_4'] == "Kesehatan Telinga, Hidung dan Tenggorokan (THT)"or row['label_5'] == "Kesehatan Telinga, Hidung dan Tenggorokan (THT)" or row['label_6'] == "Kesehatan Telinga, Hidung dan Tenggorokan (THT)"or row['label_7'] == "Kesehatan Telinga, Hidung dan Tenggorokan (THT)" or row['label_8'] == "Kesehatan Telinga, Hidung dan Tenggorokan (THT)":
    THTArray.append("1")
  else:
    THTArray.append("0")
print(len(THTArray)) 

11526


In [20]:
GigiArray = []

for index, row in machine_ant.iterrows():
  if row['label_0'] == "Gigi" or row['label_1'] == "Gigi" or row['label_2'] == "Gigi"or row['label_3'] == "Gigi" or row['label_4'] == "Gigi"or row['label_5'] == "Gigi" or row['label_6'] == "Gigi"or row['label_7'] == "Gigi" or row['label_8'] == "Gigi":
    GigiArray.append("1")
  else:
    GigiArray.append("0")
print(len(GigiArray)) 

11526


In [21]:
MataArray = []
for index, row in machine_ant.iterrows():
  if row['label_0'] == "Kesehatan Mata" or row['label_1'] == "Kesehatan Mata" or row['label_2'] == "Kesehatan Mata"or row['label_3'] == "Kesehatan Mata" or row['label_4'] == "Kesehatan Mata"or row['label_5'] == "Kesehatan Mata" or row['label_6'] == "Kesehatan Mata"or row['label_7'] == "Kesehatan Mata" or row['label_8'] == "Kesehatan Mata":
    MataArray.append("1")
  else:
    MataArray.append("0")
print(len(MataArray)) 

11526


In [22]:
BedahArray = []
for index, row in machine_ant.iterrows():
  if row['label_0'] == "Bedah" or row['label_1'] == "Bedah" or row['label_2'] == "Bedah"or row['label_3'] == "Bedah" or row['label_4'] == "Bedah"or row['label_5'] == "Bedah" or row['label_6'] == "Bedah"or row['label_7'] == "Bedah" or row['label_8'] == "Bedah":
    BedahArray.append("1")
  else:
    BedahArray.append("0")
print(len(BedahArray)) 

11526


In [23]:
KesehatanJiwaArray = []
for index, row in machine_ant.iterrows():
  if row['label_0'] == "Kesehatan Jiwa" or row['label_1'] == "Kesehatan Jiwa" or row['label_2'] == "Kesehatan Jiwa"or row['label_3'] == "Kesehatan Jiwa" or row['label_4'] == "Kesehatan Jiwa"or row['label_5'] == "Kesehatan Jiwa" or row['label_6'] == "Kesehatan Jiwa"or row['label_7'] == "Kesehatan Jiwa" or row['label_8'] == "Kesehatan Jiwa":
    KesehatanJiwaArray.append("1")
  else:
    KesehatanJiwaArray.append("0")
print(len(KesehatanJiwaArray)) 

11526


In [24]:
TulangArray = []
for index, row in machine_ant.iterrows():
  if row['label_0'] == "Ortopedi (Tulang)" or row['label_1'] == "Ortopedi (Tulang)" or row['label_2'] == "Ortopedi (Tulang)"or row['label_3'] == "Ortopedi (Tulang)" or row['label_4'] == "Ortopedi (Tulang)"or row['label_5'] == "Ortopedi (Tulang)" or row['label_6'] == "Ortopedi (Tulang)"or row['label_7'] == "Ortopedi (Tulang)" or row['label_8'] == "Ortopedi (Tulang)":
    TulangArray.append("1")
  else:
    TulangArray.append("0")
print(len(TulangArray)) 

11526


In [25]:
JantungArray = []
for index, row in machine_ant.iterrows():
  if row['label_0'] == "Jantung dan Pembuluh Darah" or row['label_1'] == "Jantung dan Pembuluh Darah" or row['label_2'] == "Jantung dan Pembuluh Darah" or row['label_3'] == "Jantung dan Pembuluh Darah" or row['label_4'] == "Jantung dan Pembuluh Darah" or row['label_5'] == "Jantung dan Pembuluh Darah" or row['label_6'] == "Jantung dan Pembuluh Darah" or row['label_7'] == "Jantung dan Pembuluh Darah" or row['label_8'] == "Jantung dan Pembuluh Darah":
    JantungArray.append("1")
  else:
    JantungArray.append("0")
print(len(JantungArray)) 

11526


In [26]:
UrologiArray = []
for index, row in machine_ant.iterrows():
  if row['label_0'] == "Urologi" or row['label_1'] == "Urologi" or row['label_2'] == "Urologi"or row['label_3'] == "Urologi" or row['label_4'] == "Urologi"or row['label_5'] == "Urologi" or row['label_6'] == "Urologi"or row['label_7'] == "Urologi" or row['label_8'] == "Urologi":
    UrologiArray.append("1")
  else:
    UrologiArray.append("0")
print(len(UrologiArray))

11526


In [27]:
SarafArray = []
for index, row in machine_ant.iterrows():
  if row['label_0'] == "Saraf" or row['label_1'] == "Saraf" or row['label_2'] == "Saraf"or row['label_3'] == "Saraf" or row['label_4'] == "Saraf"or row['label_5'] == "Saraf" or row['label_6'] == "Saraf"or row['label_7'] == "Saraf" or row['label_8'] == "Saraf":
    SarafArray.append("1")
  else:
    SarafArray.append("0")
print(len(SarafArray)) 

11526


In [28]:

ParuArray = []
for index, row in machine_ant.iterrows():
  if row['label_0'] == "Pulmonologi (Paru)" or row['label_1'] == "Pulmonologi (Paru)" or row['label_2'] == "Pulmonologi (Paru)"or row['label_3'] == "Pulmonologi (Paru)" or row['label_4'] == "Pulmonologi (Paru)"or row['label_5'] == "Pulmonologi (Paru)" or row['label_6'] == "Pulmonologi (Paru)"or row['label_7'] == "Pulmonologi (Paru)" or row['label_8'] == "Pulmonologi (Paru)":
    ParuArray.append("1")
  else:
    ParuArray.append("0")
print(len(ParuArray))

11526


In [29]:
UmumArray = []
for index, row in machine_ant.iterrows():
  if row['label_0'] == "Umum" or row['label_1'] == "Umum" or row['label_2'] == "Umum":
    UmumArray.append("1")
  else:
    UmumArray.append("0")
print(len(UmumArray))

11526


In [30]:
#masukin ke tabel machine_ant hasil encoding (BELOM DI JADIIN MODEL)
y = [KebidananArray, DalamArray, AnakArray, KulitArray, GiziArray, THTArray, GigiArray, MataArray, BedahArray, KesehatanJiwaArray, TulangArray, JantungArray, UrologiArray, SarafArray, ParuArray, UmumArray]

machine_ant["KebidananArray"] = KebidananArray
machine_ant["DalamArray"] = DalamArray
machine_ant["AnakArray"] = AnakArray
machine_ant["KulitArray"] = KulitArray
machine_ant["GiziArray"] = GiziArray
machine_ant["THTArray"] = THTArray
machine_ant["GigiArray"] = GigiArray
machine_ant["MataArray"] = MataArray
machine_ant["BedahArray"] = BedahArray
machine_ant["KesehatanJiwaArray"] = KesehatanJiwaArray
machine_ant["TulangArray"] = TulangArray
machine_ant["JantungArray"] = JantungArray
machine_ant["UrologiArray"] = UrologiArray
machine_ant["SarafArray"] = SarafArray
machine_ant["ParuArray"] = ParuArray
machine_ant["UmumArray"] = UmumArray


In [31]:
machine_ant

,id,label_0,label_1,label_2,Unnamed: 4,Unnamed: 5,label_3,label_4,label_5,label_6,...,GigiArray,MataArray,BedahArray,KesehatanJiwaArray,TulangArray,JantungArray,UrologiArray,SarafArray,ParuArray,UmumArray
0,AD-10,Kesehatan Kulit dan Kelamin,Umum,Urologi,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,1,0,0,1
1,AD-15,Bedah,Urologi,Penyakit Dalam,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,1,0,0,0,1,0,0,0
2,AD-28,Jantung dan Pembuluh Darah,Penyakit Dalam,Umum,NaN,NaN,Bedah,NaN,NaN,NaN,...,0,0,1,0,0,1,0,0,0,1
3,AD-33,Penyakit Dalam,Saraf,Kesehatan Anak,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,1,0,0
4,AD-36,Kesehatan Jiwa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11521,KD-43440,Kesehatan Mata,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,1,0,0,0,0,0,0,0,0
11522,KD-43442,Gigi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,0,0,0,0,0,0,0,0,0
11523,KD-43451,Gigi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,0,0,0,0,0,0,0,0,0
11524,KD-43460,Kesehatan Jiwa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,1,0,0,0,0,0,0


In [32]:
print("Preview testing_data")
testing.head()

Preview testing_data


,id,judul,isi
0,DS-45,kenapa telinga berdengung tidak bisa hilang ?,"selamat sore dokter , sudah 7 hari telinga say..."
1,DS-339,apabila anak anda lambat berbicara,"salam kenal team doktersehat , anak saya berum..."
2,AD-18,sakit perut setiap habis makan dan minum,"selamat malam dok , kurang lebih 2 minggu ini ..."
3,AD-66,cara mengobati vertigo berkepanjangan,selamat malam dok saya ingin brtnya seputar ve...
4,AD-73,imunisasi dapat dan polio,"malam dok , anak saya umur 5 tahun permpuan . ..."


In [33]:
testing['isi'] = testing['judul']+testing['isi']
testing

,id,judul,isi
0,DS-45,kenapa telinga berdengung tidak bisa hilang ?,kenapa telinga berdengung tidak bisa hilang ?s...
1,DS-339,apabila anak anda lambat berbicara,apabila anak anda lambat berbicarasalam kenal ...
2,AD-18,sakit perut setiap habis makan dan minum,sakit perut setiap habis makan dan minumselama...
3,AD-66,cara mengobati vertigo berkepanjangan,cara mengobati vertigo berkepanjanganselamat m...
4,AD-73,imunisasi dapat dan polio,"imunisasi dapat dan poliomalam dok , anak saya..."
...,...,...,...
2922,KD-43404,keluhan setelah berhubungan,keluhan setelah berhubungandok minggu lalu ber...
2923,KD-43406,ingin menambah tinggi badan,"ingin menambah tinggi badanselamat malam dok ,..."
2924,KD-43411,anak 1 tahun boleh dikasih nasi ?,anak 1 tahun boleh dikasih nasi ?apakah bayi y...
2925,KD-43446,mencegah infeksi virus zika,mencegah infeksi virus zikaselamat siang dok s...


In [34]:
print("Preview unlabeled_data")
unlabeled.head()

Preview unlabeled_data


,id,judul,isi
0,DS-2,mengapa lengan atas sakit walaupun tidak melak...,siang dokter mau tanya ? sudah 4 bulan lengan ...
1,DS-3,berapa batas usia kandungan untuk melakukan usg ?,batas usia kandungan untuk di usg . sampai ber...
2,DS-4,apakah katarak pada bayi harus disembuhkan den...,selamat pagi dokter . apakah katarak pada bayi...
3,DS-5,mengapa badan panas dan ngilu selama berhari-h...,nama saya ferini intan lusia umur saya 21 tahu...
4,DS-6,mengapa benjolan di muka tidak kunjung mengemp...,assalammualaikum wr.wb saya natasya umr 19 tah...


In [35]:
# Normalizing text & removing stopwords
import re

def normalize(text):
    res = text.lower().strip() # Mengubah uppercase menjadi lowercase dan melakukan trimming pada teks
    res = re.sub("\d", "", res) # Menghilangkan angka
    res = re.sub("\s+", " ", res) # Menghilangkan spasi berlebih
    res = re.sub("[^\w\s]", "", res) # Menghilangkan tanda baca
    return res

from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

stop_factory = StopWordRemoverFactory()
lst_stopword = stop_factory.get_stop_words()

# Fungsi untuk menghilangkan stopword dari suatu teks

def remove_stopword(text):
    lst_token = text.split(" ")
    res = []
    for token in lst_token:
        if token not in lst_stopword:
            res.append(token)
    return " ".join(res)

def clean_words(arr):
  corpus = []
  for text in arr:
    normalized_text = normalize(text)
    cleaned_text = remove_stopword(normalized_text)
    corpus.append(cleaned_text)
  return corpus

In [36]:
contoh_isi = contoh['isi']
contoh_isi_clean = clean_words(contoh_isi)

In [37]:
for i in range(len(contoh_isi_clean)):
  contoh['isi'][i] = contoh_isi_clean[i]

In [38]:
human_isi = human_ant['isi'].astype(str)
human_isi_clean = clean_words(human_isi)

machine_isi = machine_ant['isi'].astype(str)
machine_isi_clean = clean_words(machine_isi)

testing_isi = testing['isi'].astype(str)
testing_isi_clean = clean_words(testing_isi)

unlabeled_isi = unlabeled['isi'].astype(str)
unlabeled_isi_clean = clean_words(unlabeled_isi)

In [39]:
def replace_with_cleaned(arr,arr_clean):
  for i in range(len(arr_clean)):
    arr[i] = arr_clean[i]

In [40]:
replace_with_cleaned(human_ant['isi'],human_isi_clean)
replace_with_cleaned(machine_ant['isi'],machine_isi_clean)
replace_with_cleaned(testing['isi'],testing_isi_clean)
replace_with_cleaned(unlabeled['isi'],unlabeled_isi_clean)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [41]:
#Word Embedding using Word2Vec

# Class untuk enkapsulasi beberapa fungsionalitas yang ada

from gensim import models

class Word2VecModel:
    #Yoga
    # pretrained_file = "/content/drive/MyDrive/Colab Notebooks/Data Mining/Data TK 1 Text Mining/word2vec.id.300d.txt.gz"
    #Amanda
    pretrained_file = "word2vec.id.300d.txt.gz"
    vector_size = 300
    model = models.KeyedVectors.load_word2vec_format(pretrained_file)

    # Fungsi untuk mencari representasi vektor dari suatu kata  
    def get_word_embedding(self, word):
        try:
            return self.model[word]
        except: # Melakukan handling terhadap OOV
            return np.nan
    
    # Fungsi untuk mencari representasi vektor dari suatu kalimat
    def get_sentence_embedding(self, sent):
        lst_token = re.findall("[^\s]{3,}", sent.lower())
        rep = []
        for token in lst_token:
            try:
                token_rep = self.model[token]
                rep.append(token_rep)
            except: # Melakukan handling terhadap OOV
                continue
        return np.mean(rep, axis=0)
    
    # Fungsi untuk mengubah setiap teks pada suatu corpus menjadi representasi vektornya
    def transform(self, corpus):
        res = []
        for doc in corpus:
            tmp = self.get_sentence_embedding(doc)
            if type(tmp) == np.float64: # Melakukan handling terhadap OOV
                tmp = np.zeros(self.vector_size)
            res.append(tmp)
        return np.array(res)

In [42]:
model_word2vec = Word2VecModel()

In [43]:
#udah ada label
res_human = model_word2vec.transform(human_ant['isi'])
res_machine = model_word2vec.transform(machine_ant['isi'])
res_contoh = model_word2vec.transform(contoh['isi'])

#belom ada label
res_testing = model_word2vec.transform(testing['isi'])
res_unlabeled = model_word2vec.transform(unlabeled['isi'])

/opt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/opt/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


**BOW UNTUK VECTORIZING**

In [44]:
# # Dokumentasi: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

# from sklearn.feature_extraction.text import CountVectorizer

# vectorizer = CountVectorizer()
# res_machine = vectorizer.fit_transform(machine_ant['isi']).todense()
# res_testing = vectorizer.transform(testing['isi']).todense()
# print(len(res_testing))

In [45]:
# print(vectorizer.get_feature_names_out())

# **model bidan**

In [46]:
# Membagi dataset yang ada menjadi 2 bagian, train serta test

train_corpus_bidan, test_corpus_bidan, train_label_bidan, test_label_bidan = train_test_split(
    res_machine, 
    machine_ant['KebidananArray'], 
    test_size=0.3,
    random_state=2022
)


In [47]:
from sklearn.svm import LinearSVC
from sklearn.metrics import precision_score, f1_score, hamming_loss
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification 

classifier_svc = MLPClassifier()
classifier_svc_bidan = classifier_svc.fit(train_corpus_bidan, train_label_bidan)
#langsung ke sini kalo testing dataset
pred_res_bidan = classifier_svc_bidan.predict(test_corpus_bidan)


# **model penyakit dalam**

In [48]:
# Membagi dataset yang ada menjadi 2 bagian, train serta test

train_corpus_dalam, test_corpus_dalam, train_label_dalam, test_label_dalam = train_test_split(
    res_machine, 
    machine_ant['DalamArray'], 
    test_size=0.3,
    random_state=2022
)


In [49]:
from sklearn.svm import LinearSVC
from sklearn.metrics import precision_score, f1_score, hamming_loss

classifier_svc = MLPClassifier()
classifier_svc_dalam = classifier_svc.fit(train_corpus_dalam, train_label_dalam)
#langsung ke sini kalo testing dataset
pred_res_dalam = classifier_svc_dalam.predict(test_corpus_dalam)


# **model anak**

In [50]:
# Membagi dataset yang ada menjadi 2 bagian, train serta test

train_corpus_anak, test_corpus_anak, train_label_anak, test_label_anak = train_test_split(
    res_machine, 
    machine_ant['AnakArray'], 
    test_size=0.3,
    random_state=2022
)


In [51]:
from sklearn.svm import LinearSVC
from sklearn.metrics import precision_score, f1_score, hamming_loss

classifier_svc = MLPClassifier()
classifier_svc_anak = classifier_svc.fit(train_corpus_anak, train_label_anak)
#langsung ke sini kalo testing dataset
pred_res_anak = classifier_svc_anak.predict(test_corpus_anak)


# **model kulit kelamin**

In [52]:
# Membagi dataset yang ada menjadi 2 bagian, train serta test

train_corpus_kulit, test_corpus_kulit, train_label_kulit, test_label_kulit = train_test_split(
    res_machine, 
    machine_ant['KulitArray'], 
    test_size=0.3,
    random_state=2022
)


In [53]:
from sklearn.svm import LinearSVC
from sklearn.metrics import precision_score, f1_score, hamming_loss

classifier_svc = MLPClassifier()
classifier_svc_kulit = classifier_svc.fit(train_corpus_kulit, train_label_kulit)
#langsung ke sini kalo testing dataset
pred_res_kulit = classifier_svc_kulit.predict(test_corpus_kulit)


# **model gizi**

In [54]:
# Membagi dataset yang ada menjadi 2 bagian, train serta test

train_corpus_gizi, test_corpus_gizi, train_label_gizi, test_label_gizi = train_test_split(
    res_machine, 
    machine_ant['GiziArray'], 
    test_size=0.3,
    random_state=2022
)


In [55]:
from sklearn.svm import LinearSVC
from sklearn.metrics import precision_score, f1_score, hamming_loss

classifier_svc = MLPClassifier()
classifier_svc_gizi = classifier_svc.fit(train_corpus_gizi, train_label_gizi)
#langsung ke sini kalo testing dataset
pred_res_gizi = classifier_svc_gizi.predict(test_corpus_gizi)


# **model THT**

In [56]:
# Membagi dataset yang ada menjadi 2 bagian, train serta test

train_corpus_THT, test_corpus_THT, train_label_THT, test_label_THT = train_test_split(
    res_machine, 
    machine_ant['THTArray'], 
    test_size=0.3,
    random_state=2022
)


In [57]:
from sklearn.svm import LinearSVC
from sklearn.metrics import precision_score, f1_score, hamming_loss

classifier_svc = MLPClassifier()
classifier_svc_THT = classifier_svc.fit(train_corpus_THT, train_label_THT)
#langsung ke sini kalo testing dataset
pred_res_THT = classifier_svc_THT.predict(test_corpus_THT)


# **model Gigi**

In [58]:
# Membagi dataset yang ada menjadi 2 bagian, train serta test

train_corpus_gigi, test_corpus_gigi, train_label_gigi, test_label_gigi = train_test_split(
    res_machine, 
    machine_ant['GigiArray'], 
    test_size=0.3,
    random_state=2022
)


In [59]:
from sklearn.svm import LinearSVC
from sklearn.metrics import precision_score, f1_score, hamming_loss

classifier_svc = MLPClassifier()
classifier_svc_gigi = classifier_svc.fit(train_corpus_gigi, train_label_gigi)
#langsung ke sini kalo testing dataset
pred_res_gigi = classifier_svc_gigi.predict(test_corpus_gigi)


# **model Mata**

In [60]:
# Membagi dataset yang ada menjadi 2 bagian, train serta test

train_corpus_mata, test_corpus_mata, train_label_mata, test_label_mata = train_test_split(
    res_machine, 
    machine_ant['MataArray'], 
    test_size=0.3,
    random_state=2022
)


In [61]:
from sklearn.svm import LinearSVC
from sklearn.metrics import precision_score, f1_score, hamming_loss

classifier_svc = MLPClassifier()
classifier_svc_mata = classifier_svc.fit(train_corpus_mata, train_label_mata)
#langsung ke sini kalo testing dataset
pred_res_mata = classifier_svc_mata.predict(test_corpus_mata)


# **model Bedah**

In [62]:
# Membagi dataset yang ada menjadi 2 bagian, train serta test

train_corpus_bedah, test_corpus_bedah, train_label_bedah, test_label_bedah = train_test_split(
    res_machine, 
    machine_ant['BedahArray'], 
    test_size=0.3,
    random_state=2022
)


In [63]:
from sklearn.svm import LinearSVC
from sklearn.metrics import precision_score, f1_score, hamming_loss

classifier_svc = MLPClassifier()
classifier_svc_bedah = classifier_svc.fit(train_corpus_bedah, train_label_bedah)
#langsung ke sini kalo testing dataset
pred_res_bedah = classifier_svc_bedah.predict(test_corpus_bedah)


# **model Kesehatan Jiwa**

In [64]:
# Membagi dataset yang ada menjadi 2 bagian, train serta test

train_corpus_jiwa, test_corpus_jiwa, train_label_jiwa, test_label_jiwa = train_test_split(
    res_machine, 
    machine_ant['KesehatanJiwaArray'], 
    test_size=0.3,
    random_state=2022
)


In [65]:
from sklearn.svm import LinearSVC
from sklearn.metrics import precision_score, f1_score, hamming_loss

classifier_svc = MLPClassifier()
classifier_svc_jiwa = classifier_svc.fit(train_corpus_jiwa, train_label_jiwa)
#langsung ke sini kalo testing dataset
pred_res_jiwa = classifier_svc_jiwa.predict(test_corpus_jiwa)


# **model Tulang**

In [66]:
# Membagi dataset yang ada menjadi 2 bagian, train serta test

train_corpus_tulang, test_corpus_tulang, train_label_tulang, test_label_tulang = train_test_split(
    res_machine, 
    machine_ant['TulangArray'], 
    test_size=0.3,
    random_state=2022
)


In [67]:
from sklearn.svm import LinearSVC
from sklearn.metrics import precision_score, f1_score, hamming_loss

classifier_svc = MLPClassifier()
classifier_svc_tulang = classifier_svc.fit(train_corpus_tulang, train_label_tulang)
#langsung ke sini kalo testing dataset
pred_res_tulang = classifier_svc_tulang.predict(test_corpus_tulang)


# **model Jantung**

In [68]:
# Membagi dataset yang ada menjadi 2 bagian, train serta test

train_corpus_jantung, test_corpus_jantung, train_label_jantung, test_label_jantung = train_test_split(
    res_machine, 
    machine_ant['JantungArray'], 
    test_size=0.3,
    random_state=2022
)


In [69]:
from sklearn.svm import LinearSVC
from sklearn.metrics import precision_score, f1_score, hamming_loss

classifier_svc = MLPClassifier()
classifier_svc_jantung = classifier_svc.fit(train_corpus_jantung, train_label_jantung)
#langsung ke sini kalo testing dataset
pred_res_jantung = classifier_svc_jantung.predict(test_corpus_jantung)


# **model Urologi**

In [70]:
# Membagi dataset yang ada menjadi 2 bagian, train serta test

train_corpus_urologi, test_corpus_urologi, train_label_urologi, test_label_urologi = train_test_split(
    res_machine, 
    machine_ant['UrologiArray'], 
    test_size=0.3,
    random_state=2022
)


In [71]:
from sklearn.svm import LinearSVC
from sklearn.metrics import precision_score, f1_score, hamming_loss

classifier_svc = MLPClassifier()
classifier_svc_urologi = classifier_svc.fit(train_corpus_urologi, train_label_urologi)
#langsung ke sini kalo testing dataset
pred_res_urologi = classifier_svc_urologi.predict(test_corpus_urologi)


# **model Saraf**

In [72]:
# Membagi dataset yang ada menjadi 2 bagian, train serta test

train_corpus_saraf, test_corpus_saraf, train_label_saraf, test_label_saraf = train_test_split(
    res_machine, 
    machine_ant['SarafArray'], 
    test_size=0.3,
    random_state=2022
)


In [73]:
from sklearn.svm import LinearSVC
from sklearn.metrics import precision_score, f1_score, hamming_loss

classifier_svc = MLPClassifier()
classifier_svc_saraf = classifier_svc.fit(train_corpus_saraf, train_label_saraf)
#langsung ke sini kalo testing dataset
pred_res_saraf = classifier_svc_saraf.predict(test_corpus_saraf)


# **model Paru**

In [74]:
# Membagi dataset yang ada menjadi 2 bagian, train serta test

train_corpus_paru, test_corpus_paru, train_label_paru, test_label_paru = train_test_split(
    res_machine, 
    machine_ant['ParuArray'], 
    test_size=0.3,
    random_state=2022
)


In [75]:
from sklearn.svm import LinearSVC
from sklearn.metrics import precision_score, f1_score, hamming_loss

classifier_svc = MLPClassifier()
classifier_svc_paru = classifier_svc.fit(train_corpus_paru, train_label_paru)
#langsung ke sini kalo testing dataset
pred_res_paru = classifier_svc_paru.predict(test_corpus_paru)


# **model Umum**

In [76]:
# Membagi dataset yang ada menjadi 2 bagian, train serta test

train_corpus_umum, test_corpus_umum, train_label_umum, test_label_umum = train_test_split(
    res_machine, 
    machine_ant['UmumArray'], 
    test_size=0.3,
    random_state=2022
)


In [77]:
from sklearn.svm import LinearSVC
from sklearn.metrics import precision_score, f1_score, hamming_loss

classifier_svc = MLPClassifier()
classifier_svc_umum = classifier_svc.fit(train_corpus_umum, train_label_umum)
#langsung ke sini kalo testing dataset
pred_res_umum = classifier_svc_umum.predict(test_corpus_umum)


In [78]:
print(len(test_corpus_umum))

3458


In [79]:
print(len(pred_res_umum))

3458


# Finalisasi

URUTAN
1. Kebidanan dan Kandungan
2. Penyakit Dalam
3. Kesehatan Anak
4. Kesehatan Kulit dan Kelamin
5. Kesehatan Gizi
6. Kesehatan Telinga, Hidung dan Tenggorokan (THT)
7. Gigi
8. Kesehatan Mata
9. Bedah
10. Kesehatan Jiwa
11. Ortopedi (Tulang)
12. Jantung dan Pembuluh Darah
13. Urologi
14. Saraf
15. Pulmonologi (Paru)
16. Umum

In [80]:
testing

,id,judul,isi
0,DS-45,kenapa telinga berdengung tidak bisa hilang ?,telinga berdengung hilang selamat sore dokter ...
1,DS-339,apabila anak anda lambat berbicara,anak lambat berbicarasalam kenal team dokterse...
2,AD-18,sakit perut setiap habis makan dan minum,sakit perut habis makan minumselamat malam dok...
3,AD-66,cara mengobati vertigo berkepanjangan,mengobati vertigo berkepanjanganselamat malam ...
4,AD-73,imunisasi dapat dan polio,imunisasi poliomalam dok anak umur tahun perm...
...,...,...,...
2922,KD-43404,keluhan setelah berhubungan,keluhan berhubungandok minggu bersetubuh pasan...
2923,KD-43406,ingin menambah tinggi badan,menambah tinggi badanselamat malam dok obats...
2924,KD-43411,anak 1 tahun boleh dikasih nasi ?,anak tahun dikasih nasi bayi berumur tahun men...
2925,KD-43446,mencegah infeksi virus zika,mencegah infeksi virus zikaselamat siang dok b...


In [81]:
testing_duplicate = testing.iloc[:,[0]]

In [82]:
print(len(res_testing))

2927


In [83]:
pred_res_bidan_testing = classifier_svc_bidan.predict(res_testing)
len(pred_res_bidan_testing)

2927

In [84]:
pred_res_dalam_testing = classifier_svc_dalam.predict(res_testing)
len(pred_res_dalam_testing)

2927

In [85]:
pred_res_anak_testing = classifier_svc_anak.predict(res_testing)
len(pred_res_anak_testing)

2927

In [86]:
pred_res_kulit_testing = classifier_svc_kulit.predict(res_testing)
len(pred_res_kulit_testing)

2927

In [87]:
pred_res_gizi_testing = classifier_svc_gizi.predict(res_testing)
len(pred_res_gizi_testing)

2927

In [88]:
pred_res_THT_testing = classifier_svc_THT.predict(res_testing)
len(pred_res_THT_testing)

2927

In [89]:
pred_res_gigi_testing = classifier_svc_gigi.predict(res_testing)
len(pred_res_gigi_testing)

2927

In [90]:
pred_res_mata_testing = classifier_svc_mata.predict(res_testing)
len(pred_res_mata_testing)

2927

In [91]:
pred_res_bedah_testing = classifier_svc_bedah.predict(res_testing)
len(pred_res_bedah_testing)

2927

In [92]:
pred_res_jiwa_testing = classifier_svc_jiwa.predict(res_testing)
len(pred_res_jiwa_testing)

2927

In [93]:
pred_res_tulang_testing = classifier_svc_tulang.predict(res_testing)
len(pred_res_tulang_testing)

2927

In [94]:
pred_res_jantung_testing = classifier_svc_jantung.predict(res_testing)
len(pred_res_jantung_testing)

2927

In [95]:
pred_res_urologi_testing = classifier_svc_urologi.predict(res_testing)
len(pred_res_urologi_testing)

2927

In [96]:
pred_res_saraf_testing = classifier_svc_saraf.predict(res_testing)
len(pred_res_saraf_testing)

2927

In [97]:
pred_res_paru_testing = classifier_svc_paru.predict(res_testing)
len(pred_res_paru_testing)

2927

In [98]:
pred_res_umum_testing = classifier_svc_umum.predict(res_testing)
len(pred_res_umum_testing)

2927

In [99]:
testing_duplicate['Kebidanan'] = pred_res_bidan_testing
testing_duplicate['Dalam'] = pred_res_dalam_testing
testing_duplicate['Anak'] = pred_res_anak_testing
testing_duplicate['Kulit'] = pred_res_kulit_testing
testing_duplicate['Gizi'] = pred_res_gizi_testing
testing_duplicate['THT'] = pred_res_THT_testing

testing_duplicate['Gigi'] = pred_res_gigi_testing
testing_duplicate['Mata'] = pred_res_mata_testing
testing_duplicate['Bedah'] = pred_res_bedah_testing
testing_duplicate['KesehatanJiwa'] = pred_res_jiwa_testing

testing_duplicate['Tulang'] = pred_res_tulang_testing
testing_duplicate['Jantung'] = pred_res_jantung_testing
testing_duplicate['Urologi'] = pred_res_urologi_testing
testing_duplicate['Saraf'] = pred_res_saraf_testing
testing_duplicate['Paru'] = pred_res_paru_testing

testing_duplicate['Umum'] = pred_res_umum_testing

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [100]:
testing_duplicate

,id,Kebidanan,Dalam,Anak,Kulit,Gizi,THT,Gigi,Mata,Bedah,KesehatanJiwa,Tulang,Jantung,Urologi,Saraf,Paru,Umum
0,DS-45,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,DS-339,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,AD-18,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,AD-66,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
4,AD-73,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2922,KD-43404,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2923,KD-43406,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2924,KD-43411,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2925,KD-43446,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0


In [101]:
testing_duplicate.to_csv('databowmlpbeneran.csv', index=False, header=False)

In [102]:
arrayAccuracyScore = []
arrayF1Score = []
arrayTestLabel = [test_label_bidan,test_label_dalam,test_label_anak,test_label_kulit,test_label_gizi,test_label_THT,test_label_gigi,test_label_mata,test_label_bedah,test_label_jiwa,test_label_tulang,test_label_jantung,test_label_urologi,test_label_saraf,test_label_paru,test_label_umum]
arrayPredRes = [pred_res_bidan,pred_res_dalam,pred_res_anak,pred_res_kulit,pred_res_gizi,pred_res_THT,pred_res_gigi,pred_res_mata,pred_res_bedah,pred_res_jiwa,pred_res_tulang,pred_res_jantung,pred_res_urologi,pred_res_saraf,pred_res_paru,pred_res_umum]

for f, b in zip(arrayTestLabel, arrayPredRes):
    acc = accuracy_score(f, b)
    arrayAccuracyScore.append(acc)
    f1 = f1_score(f, b, average='macro')
    arrayF1Score.append(f1)
from statistics import mean

print('Prediksi SVC')
print('Akurasi:', mean(arrayAccuracyScore))
print('F1:', mean(arrayF1Score))

Prediksi SVC
Akurasi: 0.9159196067090805
F1: 0.782974133633638
